In [1]:
import pandas as pd
import pycountry
import numpy as np

# Motivation

We want to create a dataframe that combines the G^3 clues from the geoguessr guidebook and the travelguides.
To this end, we need a unified format:

The G^3 clues have the following format:

```
 {
    "CLUE_ID": 6,
    "text": "On occasions I have played a timed round and stumbled across a satellite dish in Brazil and its angle has helped me determine an approximate latitude.",
    "geoparsed": [
      {
        "text": "Brazil",
        "Country": "Brazil",
        "ISO3": "BRA",
        "start": 15,
        "end": 16
      }
    ]
  },
```
They are geoparsed and contain information about the countries they are applicable to.

Our word travel guide clues have the following format:

```
africa, algeria, "Summer temperatures are high throughout the country, particularly in the south where it is both very dry and very hot. During this time, road travel is difficult and air travel prone to delay because of sandstorms. Northern cities have high humidity, while those along the coast are cooled by sea breezes. In the winter, the oases of the far south are pleasant and attract many visitors. The desert temperature drops dramatically at night. North of the Sahara, temperatures are very mild from September to May and vary little between day and night. South of the Sahara, temperatures are pleasant from October to April, but there are great variations between day and night. Coastal towns are prone to storms from the sea. Rainfall is relatively low throughout the country and in the far south it is virtually unknown. Cotton and linen lightweights are best for winter months and for evenings in desert areas. Woollens and light rainwear are advised for the winter along the coast and the Hauts Plateaux. Remember that desert temperatures can drop very low during the night, so don't get caught out with only flimsy layers. South of the Sahara, from mid-December to mid-January, temperatures drop and warm clothes are necessary both in the morning and the evening. A mountain sleeping bag is also required when camping. Algeria is situated along the North African coast, bordered to the east by Tunisia and Libya, to the southeast by Niger, to the southwest by Mali, and to the west by Mauritania and Morocco. It is Africa's second-largest country with 1,200km (750 miles) of coastline. Along the coastal strip are the main towns, fertile land, beach resorts and 90% of the population. Further south lies the area of the Hauts Plateaux mountains of up to 2,000m (6,600ft) covered in cedar, pine and cypress forests with broad arable plains dividing the plateaux. The remaining 85% of the country is the Sahara Desert in its various forms, sustaining only 500,000 people, many of whom are nomadic tribes with goat and camel herds. The Sahara region contains two mountain ranges: the dramatic Hoggar massif, rising to almost 3,000m (9,800ft) and the Tassili N’Ajjer. Both are scenically stunning and offer some of the best desert landscapes in the world."

```
They contain one description for each country, and are not geoparsed yet.


At the end, we want one dataframe that combines both the guidebook and the travel guide clues. The dataframe should have the following columns:
- A unique identifer/index of the clue
- the clue itself (the text)
- the iso-2 name of the countries
- a column specifying the orgin of the clue: zero for all guidebook clues and a one for all travel guide clues

### GUIDEBOOK

The guidebook contains clues and for each clue a list of countries it is applicable to.

In [2]:
guidebook = pd.read_json("guidebook.json")

guidebook = guidebook.drop('CLUE_ID', axis=1)

remove_idx = [395, 779, 805, 3499, 3500, 3502, 3503, 3504, 3505, 3506, 3507, 3508, 3509, 3510, 3512]
guidebook = guidebook.drop(remove_idx)

guidebook['ISO3'] = None
gb_countries = []

for idx, clue in guidebook['geoparsed'].items():
    iso_list = sorted(list(set([parsed['ISO3'] for parsed in clue])))

    #while 'CHN' in iso_list:
    #    iso_list.remove('CHN')
    #while 'IND' in iso_list:
    #    iso_list.remove('IND')
    gb_countries.extend([parsed['Country'].lower().replace(' ', '-') for parsed in clue])
    if 'UAE' in iso_list:
        iso_list[iso_list.index('UAE')] = 'ARE'
    guidebook.at[idx, 'ISO3'] = iso_list
    if 'D' in iso_list:
        print([parsed['Country'].lower().replace(' ', '-') for parsed in clue])
        


guidebook = guidebook.replace('UAE', 'ARE')

iso_mapping = {} # a mapping from country name to iso3
for idx, clue in guidebook['geoparsed'].items():
    for geoparsed in clue:
        iso_mapping[geoparsed['Country'].lower().replace(' ', '-')] = geoparsed['ISO3']


guidebook.drop('geoparsed', inplace=True, axis=1)
guidebook['clue_type'] = 0

In [3]:
gb_countries = sorted(list(set(gb_countries)))

In [4]:
print(guidebook.columns)

Index(['text', 'ISO3', 'clue_type'], dtype='object')


### TRAVEL GUIDE

The travel guide contains a description for each country.

In [5]:
travel = pd.read_csv("~/Downloads/country_info_filtered.csv")
travel = travel.rename(columns={'Weather and Geography': 'text'})
travel.drop('Continent', inplace=True, axis=1)
travel = travel.dropna()
travel['clue_type'] = 1

tr_countries = sorted(list(set(travel['Country'])))

In [6]:
print(travel.columns)

Index(['Country', 'text', 'clue_type'], dtype='object')


In [7]:
country_mapping = {
    'united-states-of-america': 'united-states',
    'cape_verde': 'cabo-verde',
    'french-overseas-possessions': 'french-southern-territories',
    'palestinian-national-authority': 'palestinian-territory',
    'syrian-arab-republic': 'syria',
    'us-virgin-islands': 'u.s.-virgin-islands',
    'vatican-city': 'vatican'
}

pycountry_ios3_mapping = {}
for country in pycountry.countries:
    pycountry_ios3_mapping[country.name.lower().replace(' ', '-')] = country.alpha_3

official_pycountry_ios3_mapping  = {}
for country in pycountry.countries:
    try:
        official_pycountry_ios3_mapping[country.official_name.lower().replace(' ', '-')] = country.alpha_3
    except Exception:
        continue

manual_mapping = {
    'republic-of-congo': official_pycountry_ios3_mapping['republic-of-the-congo'],
    'democratic-republic-of-congo': official_pycountry_ios3_mapping['republic-of-the-congo'],
    'cape-verde': official_pycountry_ios3_mapping['republic-of-cabo-verde'],
    'sao-tome-e-principe': official_pycountry_ios3_mapping['democratic-republic-of-sao-tome-and-principe'],
    'tanzania': official_pycountry_ios3_mapping['united-republic-of-tanzania'],
    'bonaire': official_pycountry_ios3_mapping['bonaire,-sint-eustatius-and-saba'],
    'saba': official_pycountry_ios3_mapping['bonaire,-sint-eustatius-and-saba'],
    'st-eustatius': official_pycountry_ios3_mapping['bonaire,-sint-eustatius-and-saba'],
    'pacific-islands-of-micronesia': official_pycountry_ios3_mapping['federated-states-of-micronesia'],
    'iran': official_pycountry_ios3_mapping['islamic-republic-of-iran'],
    'surinam': official_pycountry_ios3_mapping['republic-of-suriname'],
    'st-maarten': official_pycountry_ios3_mapping['sint-maarten-(dutch-part)']    
}


In [8]:
travel['ISO3'] = None
for idx, country in travel['Country'].items():
    tmp = country.lower().replace(' ', '-')
    if tmp in country_mapping:
        tmp = country_mapping[tmp]
    #     p = True
    # else:
    #     p = False
    if tmp in pycountry_ios3_mapping:
        travel.at[idx, 'ISO3'] = [pycountry_ios3_mapping.get(tmp, None)]
    elif tmp in official_pycountry_ios3_mapping:
        travel.at[idx, 'ISO3'] = [official_pycountry_ios3_mapping.get(tmp, None)]
    elif tmp in iso_mapping:
        travel.at[idx, 'ISO3'] = [iso_mapping[tmp]]
    else:
        iso3 = [manual_mapping.get(tmp, None)]
        if None in iso3:
            continue
        travel.at[idx, 'ISO3'] = [manual_mapping.get(tmp, None)]

In [9]:
print(travel.isnull().sum().sum())
countries_with_null_iso3 = travel.loc[travel['ISO3'].isnull(), 'Country']
list(countries_with_null_iso3)

11


['brunei',
 'east-timor',
 'st-kitts-and-nevis',
 'st-lucia',
 'st-vincent-and-the-grenadines',
 'channel-islands',
 'channel-islands',
 'channel-islands',
 'channel-islands',
 'tahiti-and-her-islands',
 'falkland-islands']

In [10]:
t = travel.loc[travel['ISO3'] == 'D', 'Country']
t

Series([], Name: Country, dtype: object)

In [11]:
travel = travel.dropna()
travel.drop('Country', inplace=True, axis=1)

In [12]:
travel

,text,clue_type,ISO3
0,Summer temperatures are high throughout the co...,1,[DZA]
1,The north of the country is hot and wet during...,1,[AGO]
2,The south has an equatorial climate with four ...,1,[BEN]
3,The landlocked Botswana is semi-arid and has t...,1,[BWA]
4,Tropical. The dry season lasts from November t...,1,[BFA]
...,...,...,...
313,Subtropical with rapid changes in temperature ...,1,[PRY]
314,The weather in Peru varies according to area –...,1,[PER]
315,"Tropical and humid, cooled by the northeast tr...",1,[SUR]
316,Uruguay has an exceptionally fine temperate cl...,1,[URY]


### CLUES

In [13]:
clues = pd.DataFrame(columns=['text', 'ISO2', 'clue_type'])

In [14]:
clues = pd.concat([clues, guidebook, travel], ignore_index=True)

In [15]:
clues 

,text,ISO2,clue_type,ISO3
0,"In Street View, if the sun is clearly in the N...",NaN,0,[]
1,If the tip of a shadow is pointing south then ...,NaN,0,[]
2,The direction that the shadow’s tip points gen...,NaN,0,[]
3,If the sun is in the Northern Hemisphere then ...,NaN,0,[]
4,On some occasions the sun will appear overhead...,NaN,0,[]
...,...,...,...,...
4115,Subtropical with rapid changes in temperature ...,NaN,1,[PRY]
4116,The weather in Peru varies according to area –...,NaN,1,[PER]
4117,"Tropical and humid, cooled by the northeast tr...",NaN,1,[SUR]
4118,Uruguay has an exceptionally fine temperate cl...,NaN,1,[URY]


In [16]:
all_isos = []
clues['ISO2'] = None
for idx, iso3_list in clues['ISO3'].items():
    iso2_list = []
    if len(iso3_list) > 0:
        for iso3 in iso3_list:
            if iso3 == 'XKX':
                iso2_list.append('XK')
            elif iso3 == 'UAE':
                iso2_list.append('AE')
            else:
                iso2_list.append(pycountry.countries.get(alpha_3=iso3).alpha_2)
        all_isos.extend(iso2_list)
    clues.at[idx, 'ISO2'] = iso2_list

clues.reset_index(inplace=True)
clues = clues.rename(columns={'index': 'CLUE_ID'})

In [17]:
clues

,CLUE_ID,text,ISO2,clue_type,ISO3
0,0,"In Street View, if the sun is clearly in the N...",[],0,[]
1,1,If the tip of a shadow is pointing south then ...,[],0,[]
2,2,The direction that the shadow’s tip points gen...,[],0,[]
3,3,If the sun is in the Northern Hemisphere then ...,[],0,[]
4,4,On some occasions the sun will appear overhead...,[],0,[]
...,...,...,...,...,...
4115,4115,Subtropical with rapid changes in temperature ...,[PY],1,[PRY]
4116,4116,The weather in Peru varies according to area –...,[PE],1,[PER]
4117,4117,"Tropical and humid, cooled by the northeast tr...",[SR],1,[SUR]
4118,4118,Uruguay has an exceptionally fine temperate cl...,[UY],1,[URY]


In [18]:
clues.to_csv('./all_clues.csv', index=False)

In [19]:
all_isos = sorted(list(set(all_isos)))
encodings = np.eye(len(all_isos)).astype('int').tolist()

print(len(all_isos))
print(all_isos)

221
['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AW', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BM', 'BO', 'BQ', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CF', 'CG', 'CH', 'CI', 'CK', 'CL', 'CM', 'CN', 'CO', 'CR', 'CU', 'CV', 'CW', 'CX', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ER', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FO', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GI', 'GL', 'GM', 'GN', 'GP', 'GQ', 'GR', 'GT', 'GU', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IM', 'IN', 'IQ', 'IR', 'IS', 'IT', 'JE', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KP', 'KR', 'KW', 'KY', 'KZ', 'LA', 'LB', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MG', 'MK', 'ML', 'MM', 'MN', 'MO', 'MP', 'MQ', 'MR', 'MS', 'MT', 'MU', 'MV', 'MW', 'MX', 'MY', 'MZ', 'NA', 'NC', 'NE', 'NG', 'NI', 'NL', 'NO', 'NP', 'NR', 'NU', 'NZ', 'OM', 'PA', 'PE', 'PG', 'PH', 'PK', 'PL', 'PR', 'PS', 'PT', 'PY', 'QA', 'RE',

In [20]:
df = {
    'ISO2': all_isos,
    'encoding': encodings
    }
df = pd.DataFrame(df)

In [21]:
df

,ISO2,encoding
0,AD,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,AE,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,AF,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,AG,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,AI,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
216,XK,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
217,YE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
218,ZA,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
219,ZM,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [22]:
df.to_csv("./encodings.csv", index=False)